In [273]:
#paths required
datafolder_p = r'D:\18-DS\github\SDSHL\data\processed'
datafolder_e = r'D:\18-DS\github\SDSHL\data\external'
datafolder_i = r'D:\18-DS\github\SDSHL\data\internal'
modelfolder  = r'D:\18-DS\data\models\BERT'

# vectorfolder_bert = modelfolder + r'\BERT'
# vectorfolder_ft   = modelfolder + r'\fasttext_wiki.hi'
# vectorfolder_standford = modelfolder + r'\POS-Tagger-Hindi'

filepath_train =datafolder_p+ r'\2-train.csv'
filepath_test  =datafolder_p+ r'\2-test.csv'
filepath_fulldata = datafolder_p+ r'\2-Hinglish_Sarcasm_Clean.csv'

In [ ]:
modelforler

In [12]:
import pandas as pd
import numpy as np
import sklearn.metrics as metrics
from sklearn.naive_bayes import GaussianNB

df_train = pd.read_csv(filepath_train, sep="\t")
df_test  = pd.read_csv(filepath_test, sep="\t")
df_full  = pd.read_csv(filepath_fulldata, sep="\t", index_col="ID")

X_train =  df_train['sentence'] 
y_train = df_train['label']

X_test =  df_test['sentence'] 
y_test = df_test['label']

#import sklearn.svm as svm
from sklearn.linear_model import LogisticRegression
                
from sklearn.pipeline import Pipeline

In [3]:
prediction={}

In [ ]:
# #!pip install transformers
# Collecting transformers
#   Downloading transformers-3.1.0-py3-none-any.whl (884 kB)
# Collecting tokenizers==0.8.1.rc2
#   Downloading tokenizers-0.8.1rc2-cp38-cp38-win_amd64.whl (1.9 MB)
# Requirement already satisfied: filelock in d:\users\admin\anaconda3\lib\site-packages (from transformers) (3.0.12)
# Collecting sacremoses
#   Using cached sacremoses-0.0.43.tar.gz (883 kB)
# Requirement already satisfied: regex!=2019.12.17 in d:\users\admin\anaconda3\lib\site-packages (from transformers) (2020.6.8)
# Requirement already satisfied: numpy in d:\users\admin\anaconda3\lib\site-packages (from transformers) (1.18.5)
# Requirement already satisfied: requests in d:\users\admin\anaconda3\lib\site-packages (from transformers) (2.24.0)
# Requirement already satisfied: tqdm>=4.27 in d:\users\admin\anaconda3\lib\site-packages (from transformers) (4.47.0)
# Requirement already satisfied: packaging in d:\users\admin\anaconda3\lib\site-packages (from transformers) (20.4)
# Requirement already satisfied: sentencepiece!=0.1.92 in d:\users\admin\anaconda3\lib\site-packages (from transformers) (0.1.91)
# Requirement already satisfied: six in d:\users\admin\anaconda3\lib\site-packages (from sacremoses->transformers) (1.15.0)
# Requirement already satisfied: click in d:\users\admin\anaconda3\lib\site-packages (from sacremoses->transformers) (7.1.2)
# Requirement already satisfied: joblib in d:\users\admin\anaconda3\lib\site-packages (from sacremoses->transformers) (0.16.0)
# Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in d:\users\admin\anaconda3\lib\site-packages (from requests->transformers) (1.25.9)
# Requirement already satisfied: certifi>=2017.4.17 in d:\users\admin\anaconda3\lib\site-packages (from requests->transformers) (2020.6.20)
# Requirement already satisfied: idna<3,>=2.5 in d:\users\admin\anaconda3\lib\site-packages (from requests->transformers) (2.10)
# Requirement already satisfied: chardet<4,>=3.0.2 in d:\users\admin\anaconda3\lib\site-packages (from requests->transformers) (3.0.4)
# Requirement already satisfied: pyparsing>=2.0.2 in d:\users\admin\anaconda3\lib\site-packages (from packaging->transformers) (2.4.7)
# Building wheels for collected packages: sacremoses
#   Building wheel for sacremoses (setup.py): started
#   Building wheel for sacremoses (setup.py): finished with status 'done'
#   Created wheel for sacremoses: filename=sacremoses-0.0.43-py3-none-any.whl size=893262 sha256=819d7d410908372c75760a174f08960c8351f8776b0e67d0e9218c410bf6db62
#   Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\7b\78\f4\27d43a65043e1b75dbddaa421b573eddc67e712be4b1c80677
# Successfully built sacremoses
# Installing collected packages: tokenizers, sacremoses, transformers
# Successfully installed sacremoses-0.0.43 tokenizers-0.8.1rc2 transformers-3.1.0

## Common Functions

In [4]:
from typing import Callable, List, Optional, Tuple

import pandas as pd
from sklearn.base import TransformerMixin, BaseEstimator
import torch


class BertTransformer(BaseEstimator, TransformerMixin):
    def __init__(
            self,
            bert_tokenizer,
            bert_model,
            max_length: int = 60,
            embedding_func: Optional[Callable[[torch.tensor], torch.tensor]] = None,
    ):
        self.tokenizer = bert_tokenizer
        self.model = bert_model
        self.model.eval()
        self.max_length = max_length
        self.embedding_func = embedding_func

        if self.embedding_func is None:
            self.embedding_func = lambda x: x[0][:, 0, :].squeeze()

    def _tokenize(self, text: str) -> Tuple[torch.tensor, torch.tensor]:
        # Tokenize the text with the provided tokenizer
        tokenized_text = self.tokenizer.encode_plus(text,
                                                    add_special_tokens=True,
                                                    max_length=self.max_length
                                                    )["input_ids"]

        # Create an attention mask telling BERT to use all words
        attention_mask = [1] * len(tokenized_text)

        # bert takes in a batch so we need to unsqueeze the rows
        return (
            torch.tensor(tokenized_text).unsqueeze(0),
            torch.tensor(attention_mask).unsqueeze(0),
        )

    def _tokenize_and_predict(self, text: str) -> torch.tensor:
        tokenized, attention_mask = self._tokenize(text)

        embeddings = self.model(tokenized, attention_mask)
        return self.embedding_func(embeddings)

    def transform(self, text: List[str]):
        if isinstance(text, pd.Series):
            text = text.tolist()

        with torch.no_grad():
            return torch.stack([self._tokenize_and_predict(string) for string in text])

    def fit(self, X, y=None):
        """No fitting necessary so we just return ourselves"""
        return self

In [5]:
def pred_for_test(model):
    y_pred=[]
    for i in range(len(X_test)):
        p=model.predict(X_test.iloc[i])
        x1 = int(p[0][-1][-1])
        x2 = p[1][0]
        if x1==0: x2= 1-x2
        y_pred.append( x2 )
    return y_pred    


def print_matrics(y_pred):
    threshold=0.5
    y_pred1=[]
    for i in y_pred :
        if i>threshold:
            y_pred1.append(1)
        else:
            y_pred1.append(0)

    acc = np.round( metrics.accuracy_score(y_test,y_pred1), 2)
    recall = np.round( metrics.recall_score(y_test,y_pred1), 2)
    precision = np.round( metrics.precision_score(y_test,y_pred1), 2)
    f1 = np.round( metrics.f1_score(y_test,y_pred1), 2)
    roc = np.round( metrics.roc_auc_score(y_test,y_pred1), 2)

    print("Accuracy : ", acc )
    print("Recall   : ", recall )
    print("Precision: ", precision )
    print("F1       : ", f1 )
    print("ROC      : ", roc )

# mBERT for SDSHL Transfer Learning - Task

In [6]:
from transformers import BertConfig, BertModel, BertTokenizer

In [ ]:
# Download model and configuration from S3 and cache.
# bert_model_ml = BertModel.from_pretrained('bert-base-multilingual-uncased')
# Model was saved using `save_pretrained('./test/saved_model/')` (for example purposes, not runnable).
#model = BertModel.from_pretrained('')
# Update configuration during loading.
#model = BertModel.from_pretrained('bert-base-uncased', output_attention=True)
#assert model.config.output_attention == True
# Loading from a TF checkpoint file instead of a PyTorch model (slower, for example purposes, not runnable).
#config = BertConfig.from_json_file('./tf_model/my_tf_model_config.json')
#model = BertModel.from_pretrained('./tf_model/my_tf_checkpoint.ckpt.index', from_tf=True, config=config)

In [278]:
bert_model_ml     = BertModel.from_pretrained(modelfolder + r'\bert-base-multilingual-uncased')
bert_tokenizer_ml = BertTokenizer.from_pretrained(modelfolder + r'\bert-base-multilingual-uncased')

# bert_model_ml     = BertModel.from_pretrained('bert-base-multilingual-uncased')
# bert_tokenizer_ml = BertTokenizer.from_pretrained('bert-base-multilingual-uncased',".")

#train_tokens = list(map(lambda t: ['[CLS]'] + bert_tokenizer_ml.tokenize(t)[:511], x_train))
#test_tokens = list(map(lambda t: ['[CLS]'] + bert_tokenizer_ml.tokenize(t)[:511], x_test))



In [282]:
type(bert_model_ml), type(bert_tokenizer_ml)

(pytorch_pretrained_bert.modeling.BertModel,
 pytorch_pretrained_bert.tokenization.BertTokenizer)

In [279]:
# train_tokens_ids = list(map(bert_tokenizer_ml.convert_tokens_to_ids, train_tokens))
# train_tokens_ids = pad_sequences(train_tokens_ids, maxlen=512, truncating="post", padding="post", dtype="int")

# test_tokens_ids = list(map(bert_tokenizer_ml.convert_tokens_to_ids, test_tokens))
# test_tokens_ids = pad_sequences(test_tokens_ids, maxlen=512, truncating="post", padding="post", dtype="int")


bert_transformer = BertTransformer(bert_tokenizer_ml, bert_model_ml, max_length=200)
#classifier = svm.LinearSVC(C=1.0, class_weight="balanced")
#classifier = LogisticRegression()
classifier = GaussianNB()

model_pipeline_ml = Pipeline(
    [
        ("vectorizer", bert_transformer),
        ("classifier", classifier),
    ]
)
model_pipeline_ml.fit(x_train, y_train)

AttributeError: 'BertTokenizer' object has no attribute 'encode_plus'

In [16]:
y_pred = model_pipeline_ml.predict_proba(x_test)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [19]:
y_pred1=model_pipeline_ml.predict(x_test)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [26]:
#y_pred = pred_for_test(bert_model_ml)
prediction['bert_multiling_pretrained'] = y_pred[:,1]
print_matrics(y_pred[:,1])

Accuracy :  0.64
Recall   :  0.7
Precision:  0.62
F1       :  0.66
ROC      :  0.64


In [ ]:
# acc = np.round( metrics.accuracy_score(y_test,y_pred), 2)
# recall = np.round( metrics.recall_score(y_test,y_pred), 2)
# precision = np.round( metrics.precision_score(y_test,y_pred), 2)
# f1 = np.round( metrics.f1_score(y_test,y_pred), 2)
# roc = np.round( metrics.roc_auc_score(y_test,y_pred), 2)

# print("Accuracy : ", acc )
# print("Recall   : ", recall )
# print("Precision: ", precision )
# print("ROC      : ", f1 )
# print("roc      : ", roc )

# <font color=green> Indicnlp BERT<font>

### Indicnlp for SDSHL Transfer Learning - Embedding

In [28]:
from transformers import AutoTokenizer, AutoModel

bert_tokenizer_indic = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")

bert_model_indic = AutoModel.from_pretrained("ai4bharat/indic-bert")

In [29]:

bert_transformer = BertTransformer(bert_tokenizer_indic, bert_model_indic, max_length=200)
#classifier = svm.LinearSVC(C=1.0, class_weight="balanced")
#classifier = LogisticRegression()
classifier = GaussianNB()

model_pipeline = Pipeline(
    [
        ("vectorizer", bert_transformer),
        ("classifier", classifier),
    ]
)
model_pipeline.fit(x_train, y_train)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

...

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Pipeline(steps=[('vectorizer',
                 BertTransformer(bert_model=None, bert_tokenizer=None,
                                 embedding_func=<function BertTransformer.__init__.<locals>.<lambda> at 0x000002E342E038B0>,
                                 max_length=200)),
                ('classifier', GaussianNB())])

In [ ]:
#y_pred = pred_for_test(bert_model_indic)
y_pred = model_pipeline.predict_proba(x_test)

In [35]:
prediction['bert_indicnlp_pretrained'] = y_pred[:,1]
print_matrics(y_pred[:,1])

Accuracy :  0.57
Recall   :  0.68
Precision:  0.56
F1       :  0.62
ROC      :  0.58


In [33]:
df_prediction= pd.DataFrame(prediction, index=df_test.index)
df_prediction.to_csv(datafolder_p+ r'\model_predictions_bert.csv')
df_prediction

,bert_multiling_pretrained,bert_indicnlp_pretrained
0,1.000000e+00,1.000000e+00
1,1.000000e+00,1.000000e+00
2,9.937309e-01,1.000000e+00
3,1.162802e-21,9.999397e-01
4,8.641040e-01,9.999785e-01
...,...,...
195,9.999999e-01,2.210688e-05
196,9.845037e-01,1.000000e+00
197,2.089919e-24,3.071928e-14
198,5.212929e-05,9.976244e-01


# Convert Sentences to Vector & save Embedding

### mBERT for SDSHL Transfer Learning - Embedding

In [ ]:
#This steps takes approx 5 min.

import torch

mbert_embedding=[]
for i in range(len(df_full)):
    txt = df_full.iloc[i,:]['sentence']
    token_dict = bert_tokenizer_ml.encode_plus(txt, add_special_tokens=False, max_length=100, truncation=True)
    embed = bert_model_ml((torch.tensor(token_dict["input_ids"]).unsqueeze(0)))
    embed = embed[1][0].detach().numpy()
    #print (i, sep='\t')
    mbert_embedding.append(embed)

In [ ]:
df_mbert = pd.DataFrame(mbert_embedding, index=df_full.index)
df_mbert['label'] = df_full['label']
df_mbert.head(5)

In [ ]:
df_mbert.to_csv(datafolder_p+r'\embedding_bert768_mbert_pretrained.csv')

### indicnlp bert embedding

In [ ]:
#This steps takes approx 5 min.

indicbert_embedding=[]
for i in range(len(df_full)):
    txt = df_full.iloc[i,:]['sentence']
    token_dict = tokenizer.encode_plus(txt, add_special_tokens=False, max_length=100, truncation=True)
    embed = model((torch.tensor(token_dict["input_ids"]).unsqueeze(0)))
    embed = embed[1][0].detach().numpy()
    #print (i, sep='\t')
    indicbert_embedding.append(embed)

In [ ]:
df_indicbert = pd.DataFrame(indicbert_embedding, index=df_full.index)
df_indicbert['label'] = df_full['label']
df_indicbert.head(5)

In [ ]:
df_indicbert.to_csv(datafolder_p+r'\embedding_bert768_indicnlp_pretrained.csv')

# <font color=red> BERT Experiments from a Blog</font>
https://towardsdatascience.com/bert-to-the-rescue-17671379687f <br>
https://github.com/shudima/notebooks/blob/master/BERT_to_the_rescue.ipynb

In [238]:
train_texts = list(X_train)
train_labels = list(y_train)

test_texts = list(X_test)
test_labels = list(y_test)

len(train_texts), len(train_labels), len(test_texts), len(test_labels)

(1800, 1800, 200, 200)

In [159]:
!pip install pytorch_pretrained_bert pytorch-nlp

  Using cached pytorch_pretrained_bert-0.6.2-py3-none-any.whl (123 kB)


In [233]:
import sys
import numpy as np
import random as rn
import torch
from pytorch_pretrained_bert import BertModel
from torch import nn
from torchnlp.datasets import imdb_dataset
from pytorch_pretrained_bert import BertTokenizer
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_
from IPython.display import clear_output

In [234]:
rn.seed(321)
np.random.seed(321)
torch.manual_seed(321)
torch.cuda.manual_seed(321)

In [236]:
# train_data, test_data = imdb_dataset(train=True, test=True)
# rn.shuffle(train_data)
# rn.shuffle(test_data)
# train_data = train_data[:1000]
# test_data = test_data[:100]

In [237]:
# train_texts, train_labels = list(zip(*map(lambda d: (d['text'], d['sentiment']), train_data)))
# test_texts, test_labels = list(zip(*map(lambda d: (d['text'], d['sentiment']), test_data)))

# len(train_texts), len(train_labels), len(test_texts), len(test_labels)

(1000, 1000, 100, 100)

In [239]:
tokenizer_multi    = BertTokenizer.from_pretrained(r'D:\18-DS\data\models\BERT\bert-base-multilingual-uncased', do_lower_case=True)
tokenizer_indicnlp = BertTokenizer.from_pretrained(r'D:\18-DS\data\models\BERT\bert_indicnlp_pretrained', do_lower_case=True)



100%|███████████████████████████████████████████████████████████████████████| 871891/871891 [00:03<00:00, 266453.32B/s]


In [241]:
tokenizer.tokenize('वायरस का कहर अभी खत्म नहीं हुआ है घर से उतना ही निकलें , जितना चुनाव के बाद राहुल मंदिर')

['वा',
 '##यर',
 '##स',
 'का',
 'क',
 '##हर',
 'अ',
 '##भी',
 'ख',
 '##तम',
 'नही',
 'हआ',
 'ह',
 'घर',
 'स',
 'उ',
 '##तन',
 '##ा',
 'ही',
 'न',
 '##िकल',
 ',',
 'ज',
 '##ित',
 '##ना',
 'चनाव',
 'क',
 'बाद',
 'र',
 '##ाह',
 '##ल',
 'मदिर']

In [242]:

train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:510] + ['[SEP]'], train_texts))
test_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:510] + ['[SEP]'], test_texts))

len(train_tokens), len(test_tokens)

(1800, 200)

In [243]:
train_tokens[0]

['[CLS]',
 'ल',
 '##ो',
 'इसकी',
 'तो',
 'कोई',
 'सन',
 '##ता',
 'ही',
 'नही',
 'सर',
 '##दार',
 'प',
 '##टल',
 'स',
 '##ाब',
 'न',
 'ज',
 '##ित',
 '##ना',
 'स',
 '##मम',
 '##ान',
 'प',
 '##टल',
 'जाति',
 'को',
 'दिल',
 '##ाया',
 'उ',
 '##तन',
 '##ा',
 'ही',
 'इस',
 'दसर',
 'प',
 '##टल',
 'न',
 'ग',
 '##वा',
 'दिया',
 '[SEP]']

In [246]:
train_tokens_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids, train_tokens)), maxlen=256, truncating="post", padding="post", dtype="int")
test_tokens_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids, test_tokens)), maxlen=256, truncating="post", padding="post", dtype="int")

train_tokens_ids.shape, test_tokens_ids.shape

((1800, 256), (200, 256))

In [247]:
train_tokens_ids[0]

array([  101,   574, 11231, 48638, 21426, 37955, 15518, 13142, 14473,
       17024, 47505, 46027,   567, 48149,   579, 80969,   566,   554,
       13036, 17068,   579, 76268, 16985,   567, 48149, 52460, 11483,
       88607, 37524,   536, 37232, 10949, 14473, 13901, 52254,   567,
       48149,   566,   549, 27185, 20574,   102,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [249]:
train_y = np.array(train_labels) == 1
test_y = np.array(test_labels) == 1
train_y.shape, test_y.shape, np.mean(train_y), np.mean(test_y)

((1800,), (200,), 0.5, 0.5)

In [250]:
train_masks = [[float(i > 0) for i in ii] for ii in train_tokens_ids]
test_masks = [[float(i > 0) for i in ii] for ii in test_tokens_ids]

In [225]:
#train_masks[100]

### Baseline

In [251]:
baseline_model = make_pipeline(CountVectorizer(ngram_range=(1,3)), LogisticRegression()).fit(train_texts, train_labels)

In [252]:
baseline_predicted = baseline_model.predict(test_texts)

In [253]:
print(classification_report(test_labels, baseline_predicted))

              precision    recall  f1-score   support

           0       0.66      0.73      0.70       100
           1       0.70      0.63      0.66       100

    accuracy                           0.68       200
   macro avg       0.68      0.68      0.68       200
weighted avg       0.68      0.68      0.68       200



### Finetune

In [271]:
class BertBinaryClassifier(nn.Module):
    def __init__(self, dropout=0.1, bert_type='multi'):
        super(BertBinaryClassifier, self).__init__()
        if bert_type=="multi":
            self.bert = BertModel.from_pretrained(r'D:\18-DS\data\models\BERT\bert-base-multilingual-uncased',)
        else:
            self.bert = BertModel.from_pretrained(r'D:\18-DS\data\models\BERT\bert_indicnlp_pretrained')

        #self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokens, masks=None):
        _, pooled_output = self.bert(tokens, attention_mask=masks, output_all_encoded_layers=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        proba = self.sigmoid(linear_output)
        return proba

In [255]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [256]:
device

device(type='cpu')

In [272]:
bert_clf = BertBinaryClassifier("multi")
#bert_clf = bert_clf.cuda()
bert_clf = bert_clf.cpu()

In [258]:
BATCH_SIZE = 4
EPOCHS = 10

In [259]:
train_tokens_tensor = torch.tensor(train_tokens_ids)
train_y_tensor = torch.tensor(train_y.reshape(-1, 1)).float()

test_tokens_tensor = torch.tensor(test_tokens_ids)
test_y_tensor = torch.tensor(test_y.reshape(-1, 1)).float()

train_masks_tensor = torch.tensor(train_masks)
test_masks_tensor = torch.tensor(test_masks)

#str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

In [260]:

train_dataset = TensorDataset(train_tokens_tensor, train_masks_tensor, train_y_tensor)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

test_dataset = TensorDataset(test_tokens_tensor, test_masks_tensor, test_y_tensor)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

In [261]:
param_optimizer = list(bert_clf.sigmoid.named_parameters()) 
optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

In [262]:
optimizer = Adam(bert_clf.parameters(), lr=3e-6)

In [263]:
torch.cuda.empty_cache()

In [264]:
for epoch_num in range(EPOCHS):
    bert_clf.train()
    train_loss = 0
    for step_num, batch_data in enumerate(train_dataloader):
        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)
        #print(str(torch.cuda.memory_allocated(device)/1000000 ) + 'M')
        logits = bert_clf(token_ids, masks)
        
        loss_func = nn.BCELoss()

        batch_loss = loss_func(logits, labels)
        train_loss += batch_loss.item()
        
        
        bert_clf.zero_grad()
        batch_loss.backward()
        

        clip_grad_norm_(parameters=bert_clf.parameters(), max_norm=1.0)
        optimizer.step()
        
        clear_output(wait=True)
        print('Epoch: ', epoch_num + 1)
        print("\r" + "{0}/{1} loss: {2} ".format(step_num, len(train_data) / BATCH_SIZE, train_loss / (step_num + 1)))

RuntimeError: Expected tensor for argument #1 'indices' to have scalar type Long; but got torch.IntTensor instead (while checking arguments for embedding)

In [ ]:
bert_clf.eval()
bert_predicted = []
all_logits = []
with torch.no_grad():
    for step_num, batch_data in enumerate(test_dataloader):

        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)

        logits = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        loss = loss_func(logits, labels)
        numpy_logits = logits.cpu().detach().numpy()
        
        bert_predicted += list(numpy_logits[:, 0] > 0.5)
        all_logits += list(numpy_logits[:, 0])

In [ ]:
np.mean(bert_predicted)

In [ ]:
print(classification_report(test_y, bert_predicted))

# <font color=red> BERT English Twitter Experiments</font>

In [ ]:
#model.save_pretrained(vectorfolder+r'\bert-base-multilingual-uncased')

In [ ]:
bert_tokenizer_ml = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
inputs

In [ ]:
from transformers import BertTokenizer, BertForNextSentencePrediction
import torch

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_ns_model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased', return_dict=True)

In [ ]:
from transformers import pipeline

In [ ]:
classifier_pl_ml = pipeline('sentiment-analysis', model='bert-base-multilingual-uncased')

results = classifier_pl_ml(["We are very happy to show you the 🤗 Transformers library.",
                      "We hope you don't hate it.",
                     "I love transformers",
                     "What kind of works is this?",
                     "मुझे राजनीति पसंद नही है",
                     "मुझे पढना अच्छा लगता है"])
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

In [ ]:
classifierSA_pl_ml = pipeline('sentiment-analysis', model="nlptown/bert-base-multilingual-uncased-sentiment")

results = classifierSA_pl_ml(["We are very happy to show you the 🤗 Transformers library.",
                      "We hope you don't hate it.",
                     "I love transformers",
                     "What kind of works is this?",
                     "मुझे राजनीति पसंद नही है",
                     "मुझे पढना अच्छा लगता है"])
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

In [ ]:
#model.num_parameters
#model.base_model
#model.embeddings
#model.init_weights
# model.num_parameters
# model.modules

In [ ]:
from transformers import BertModel, BertConfig

# Initializing a BERT bert-base-uncased style configuration
configuration = BertConfig()

# Initializing a model from the bert-base-uncased style configuration
model3 = BertModel(configuration)

# Accessing the model configuration
#configuration = model.config

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM

import torch

tokenizer1 = BertTokenizer.from_pretrained('bert-base-uncased')
model1 = BertForMaskedLM.from_pretrained('bert-base-uncased')

input_ids = tokenizer("Hello, my dog is cute", return_tensors="pt")["input_ids"]
input_ids = tokenizer("निर्धन गिरे पहाड़ न पूछे हाल करोड़पति कांटा लगे पूछे लोग हजार", return_tensors="pt")["input_ids"]

outputs = model1(input_ids, labels=input_ids)
loss, prediction_scores = outputs[:2]

In [ ]:
input_ids

# BERT Experiments from a Blog

### https://towardsdatascience.com/build-a-bert-sci-kit-transformer-59d60ddd54a5

In [ ]:
file = r'D:\18-DS\data\Datasets_nlp\text_emotion.csv'
import numpy as np

In [ ]:
from typing import Callable, List, Optional, Tuple

import pandas as pd
from sklearn.base import TransformerMixin, BaseEstimator
import torch


class BertTransformer(BaseEstimator, TransformerMixin):
    def __init__(
            self,
            bert_tokenizer,
            bert_model,
            max_length: int = 60,
            embedding_func: Optional[Callable[[torch.tensor], torch.tensor]] = None,
    ):
        self.tokenizer = bert_tokenizer
        self.model = bert_model
        self.model.eval()
        self.max_length = max_length
        self.embedding_func = embedding_func

        if self.embedding_func is None:
            self.embedding_func = lambda x: x[0][:, 0, :].squeeze()

    def _tokenize(self, text: str) -> Tuple[torch.tensor, torch.tensor]:
        # Tokenize the text with the provided tokenizer
        tokenized_text = self.tokenizer.encode_plus(text,
                                                    add_special_tokens=True,
                                                    max_length=self.max_length
                                                    )["input_ids"]

        # Create an attention mask telling BERT to use all words
        attention_mask = [1] * len(tokenized_text)

        # bert takes in a batch so we need to unsqueeze the rows
        return (
            torch.tensor(tokenized_text).unsqueeze(0),
            torch.tensor(attention_mask).unsqueeze(0),
        )

    def _tokenize_and_predict(self, text: str) -> torch.tensor:
        tokenized, attention_mask = self._tokenize(text)

        embeddings = self.model(tokenized, attention_mask)
        return self.embedding_func(embeddings)

    def transform(self, text: List[str]):
        if isinstance(text, pd.Series):
            text = text.tolist()

        with torch.no_grad():
            return torch.stack([self._tokenize_and_predict(string) for string in text])

    def fit(self, X, y=None):
        """No fitting necessary so we just return ourselves"""
        return self

In [ ]:
txt = '''लोग वतन तक खा जाते हैं इसका इसे यकीन नहींमान जाएगा तू ले जाकर दिल्ली इसे दिखा ला दोस्त
अंग्रेजी नहीं आती है इसलिए हिन्दी ट्विट ज्यादा करता हूँ . निजी राय बुरा लगे तो भाड़ मे जा .
कश्मीर में हो रहा है जल जिहाद .
संघर्ष में आदमी अकेला होता है सफलता में दुनिया उसके साथ होती है जब जब जग उस पर हसा है तब तब उसी ने नया इतिहास रचा है.
मोहब्बत अंधी होती है
तीखी बात में राम जेठमलानी ने गड्गरी पर तो इल्जाम लगा दिए जब की किसी सरकरी संस्था ने अभी इल्जाम नही लगया इतनी जिंदगी देखने पर आदमी का अहम्
एक मुलाकात जरूरी है रिहा होने के लिये .एक उम्र से कैद हूँ आँसू की तरह .
अगर मुलायम और अखिलेश चाहें तो UP में एक दंगा और करवा के बिल महेंद्र सिंह धोनी के नाम पे फाड़ सकते हैं. #IndvsEng
जब तन्हा रातो में मोगैम्बो चिलाता है की मोगैम्बो खुश हुवा तो समझिये की मनमोहन सरकार के बुरे सपने याद आ गये
काटजू को 1993 में मारे गए लोगों से कोई सहानुभूति नहीं है और उसका मूंह केवल और केवल कांग्रेस तथा इसके द्वारा समर्थित लोगो के लिए ही खुलता है.
कुछ ऐसे भी हादसे होते है जिंदगी में इंसान बच तो जाता है पर ज़िंदा नहीं रहता शुभ रात्रि मित्रों 😁
#News तीन साल बाथरूम में बंद रही विवाहिता
तीन साल बाथरूम में बंद रही विवाहिता दहेज प्रताड़ना शिकार बिहार के दरभंगा जिला निवासी गुंजा तीन साल बाद रविवार
क़सम से युपीए 3 आई तो मै भी कांग्रेसी बन जांऊगा साला कब तक ईमानदार बन अपना ही घर जलाऊँगा ?
पवन और अश्विनी का इस्तीफा होना ही नहीं था आज चोर ग्रुप में डिसिशन इसलिए हुआ की कर्णाटक का चुनाव था जिससे क्रेडिबिलिटी वोट पर असर ना पड़े
बुला कर तुमने महफ़िल में हमको गैरों से उठवाया हमीं खुद उठ गए होते इशारा कर दिया होता NaushadLakhnavi
#BJP की संवैधानिक तिकड़म संविधान की धारा का गलत हवाला देकर सीक्रेट बैलट ( जो होता ही नहीं ) से दिल्ली में सरकार बनाने की अलोकतांत्रिक साजिश.
मुझे ये बयान पर कोई शिकायत नहीं है . हम को शिकायत है तो केंद्र सरकार के गृह मंत्रालय से है सुप्रीम कोर्ट से है .
जम्मू कश्मीर भूस्खलन होने की वजह से 7 लोगो की मौत हो गई जबकि 20 लोग अब भी लापता है.
अब सभी ताकतें एक दुसरे से जुदा हैं और आपस में बर्चस्व की लड़ाई लड़ रहे हैं.
जो जम्मू कश्मीर में हो रहा है उसे जल जिहाद कहते हैं.
खुदा को भूल उन्हीं के आगे दामन फैलाया जिनके मरने की दुआएं मांगी थी खुदा से
हुर्रियत जेहादी यासिन मलिक और आजाद कश्मीर चिलाने वाले कश्मीर को बचाने आगे नहीं आएंगे #JammuKashmirFloods. #KashmirFloods
मन्नूजी पिछले 10 सालो में संविधान की पूजा ही उतारते रहे है ? कांग्रेसी लोग कितने बेशर्म है.'''

In [ ]:
# tokenized_dict = tokenizer.encode_plus(
#     txt,
#     add_special_tokens=True,
#     max_length=400
#     )

In [ ]:
# #tokenized_text
# tokenized_dict['input_ids'].unsqeeze(0)

In [ ]:
# #bert_model(torch.tensor(tokenized_dict['input_ids'].unsqueeze(0)))

# bert_model(torch.tensor(tokenized_text.unsqueeze(0)))

In [ ]:
#torch.tensor(tokenized_dict['input_ids'])

In [ ]:
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# tokenized_dict = tokenizer.encode_plus(
#     "hi my name is nicolas",
#     add_special_tokens=True,
#     max_length=5
#     )

In [ ]:
# tok_dict = tokenizer.encode_plus( 'This is my place You are most welcomde', add_special_tokens=True, max_length=100)
# tok_dict

In [ ]:
# torch.tensor(tok_dict["input_ids"])

In [ ]:
bert_model==mod

In [ ]:
#bert_model(torch.tensor(tokenized_text.unsqueeze(0)))

In [ ]:
bert_model = BertModel.from_pretrained("bert-base-uncased")
tokenized_text = torch.tensor(tokenized_dict["input_ids"])
with torch.no_grad():
    embeddings = bert_model(torch.tensor(tokenized_text.unsqueeze(0)))

In [ ]:
#import pickle
#filename='bert_twitter_emotions_en.pkl'

In [ ]:
#!pip install joblib

# #from sklearn.externals import joblib
# import joblib
# # Save the model as a pickle in a file 
# joblib.dump(model, filename) 

In [ ]:
figure8_df = pd.read_csv(file)
split = np.random.choice(
    ["train", "val", "test"],
    size=figure8_df.shape[0],
    p=[.7, .15, .15]
)
figure8_df["split"] = split
x_train = figure8_df[figure8_df["split"] == "train"]
y_train = x_train["sentiment"]

In [ ]:
import sklearn.svm as svm
from sklearn.pipeline import Pipeline

In [ ]:
bert_transformer = BertTransformer(bert_tokenizer, bert_model)
classifier = svm.LinearSVC(C=1.0, class_weight="balanced")
model_pipeline = Pipeline(
    [
        ("vectorizer", bert_transformer),
        ("classifier", classifier),
    ]
)
model_pipeline.fit(x_train["content"], y_train)

In [ ]:
bert_model.save_pretrained('bert_twitter_emotions_en')

In [ ]:
mod= BertModel.from_pretrained(r'D:\18-DS\github\SDSHL\notebooks\experiments\bert_twitter_emotions_en')

In [ ]:
# from datetime import datetime

# # datetime object containing current date and time
# now = datetime.now()
 
# print("now =", now)

# # dd/mm/YY H:M:S
# dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
# print("date and time =", dt_string)	

In [ ]:
# from sklearn.feature_extraction.text import (
#    CountVectorizer, TfidfTransformer
# )

# tf_idf = Pipeline([
#     ("vect", CountVectorizer()),
#     ("tfidf", TfidfTransformer())
#     ])

# model = Pipeline([
#     ("union", FeatureUnion(transformer_list=[
#         ("bert", bert_transformer),
#         ("tf_idf", tf_idf)
#         ])),
#         ("classifier", classifier),
#     ])

In [ ]:
x_test = figure8_df[figure8_df["split"] == "test"]
y_test = x_test["sentiment"]

y_pred = model.predict(x_test['content'])

In [ ]:
import sklearn.metrics as metrics

In [ ]:
metrics.confusion_matrix(y_test,y_pred)

In [ ]:
metrics.accuracy_score(y_test,y_pred)

In [ ]:
list(zip(y_test,y_pred))